In [1]:
import requests
from pprint import pprint
import pandas as pd
import numpy as np
from faker import Faker
from tqdm import tqdm
import random

In [2]:
# queries based on gender
women_queries = [ ["red", "wedding", "lehenga"],
            ["black", "maxi", "dress"], 
            ["blue", "saree", "indian"],
            ["striped", "crop", "top", "women"],
            ["denim", "jeans", "grey"],
            ["knee-length", "boots", "brown"],
            ["white", "sandals", "fancy"],
            ["traditional", "jhumkas"],
            ["black", "sunglasses", "aviator"],
            ["party", "bag", "night"]
]

men_queries = [ ["red", "wedding", "sherwani"],
            ["black", "tuxedo", "suit"],
            ["blue", "kurta", "indian"],
            ["denim", "jeans", "grey"],
            ["boots", "brown"],
            ["traditional", "earrings", "male"],
            ["black", "sunglasses", "aviator"],
            ["party", "bag", "night"]
]

In [5]:
# generate product datasets for women
products_women = []
for query in women_queries:
    string = " ".join(query)
    url = "https://flipkart-scraper-api.dvishal485.workers.dev/search/q=" + string
    data = requests.get(url).json()["result"]
    products_women.extend(data)



In [6]:
products_women

[{'name': 'Embroidered Semi Stitched Lehenga &amp; Crop Top',
  'link': 'https://www.flipkart.com/matrubhumi-embroidered-semi-stitched-lehenga-crop-top/p/itm2e561b2a80c5a',
  'current_price': 270,
  'original_price': 1000,
  'discounted': True,
  'thumbnail': 'https://rukminim2.flixcart.com/image/612/612/khwbde80-0/lehenga-choli/6/m/e/free-half-sleeve-parita-a1-aasmaan-original-imafxt5zzytrd9hc.jpeg?q=70',
  'query_url': 'https://flipkart-scraper-api.dvishal485.workers.dev/product/matrubhumi-embroidered-semi-stitched-lehenga-crop-top/p/itm2e561b2a80c5a'},
 {'name': 'Embroidered, Embellished, Self Design, Chevron/Zig Zag ...',
  'link': 'https://www.flipkart.com/crissy-embroidered-embellished-self-design-chevron-zig-zag-semi-stitched-lehenga-choli/p/itm7b2b36fcf87d4',
  'current_price': 899,
  'original_price': 3599,
  'discounted': True,
  'thumbnail': 'https://rukminim2.flixcart.com/image/612/612/l45xea80/lehenga-choli/e/d/g/free-sleeveless-ruffle-lehenga-pmd-fashion-original-imagf4kj

In [7]:
# extract product details for product listed
products_women_details = []
idx = 0
for product in tqdm(products_women):
    url = product["query_url"]
    res = requests.get(url)
    products_women_details.append(res.json())


100%|██████████| 400/400 [14:33<00:00,  2.18s/it]


In [10]:
# save the dataset as a dataframe and pickle file
products_women_details = pd.DataFrame(products_women_details)
products_women_details.to_pickle("products_women_details.pkl")

In [3]:
# same process repeated for products dataset for men
products_men = []
for query in men_queries:
    string = " ".join(query)
    url = "https://flipkart-scraper-api.dvishal485.workers.dev/search/q=" + string
    data = requests.get(url).json()["result"]
    products_men.extend(data)



In [4]:
products_men_details = []
idx = 0
for product in tqdm(products_men):
    url = product["query_url"]
    res = requests.get(url)
    products_men_details.append(res.json())


  0%|          | 0/319 [00:00<?, ?it/s]

100%|██████████| 319/319 [12:09<00:00,  2.29s/it]


In [5]:
products_men_details = pd.DataFrame(products_men_details)
products_men_details.to_pickle("products_men_details.pkl")

In [6]:
# read the products from the pickle files
product_women_details = pd.read_pickle("products_women_details.pkl")
product_men_details = pd.read_pickle("products_men_details.pkl")

In [7]:
# list of columns that are used to create the dataset
Columns = ["UserID", "FirstName", "LastName", "Age", "Gender", "Country", "ProductsBought", "ProductsViewedInLast30Days", "ProductsInWishlist"]

In [9]:
# create users dataset
fixed_digits = 6
users = pd.DataFrame(columns = Columns)
Gender = ["F", "M"]
for i in tqdm(range(100)):
    fake = Faker()
    data = {}
    id = random.randrange(111111, 999999, fixed_digits)
    while id in users["UserID"]:
        id = random.randrange(111111, 999999, fixed_digits)
    data["UserID"] = str(id)
    data["LastName"] = fake.last_name()
    data["Age"] = random.randint(15, 80)
    data["Gender"] = random.choice(Gender)
    data["Country"] = fake.country()

    products_bought = []
    products_viewed = []
    products_wishlist = []

    if data["Gender"] == "F":
        product_details = product_women_details
    else:
        product_details = product_men_details
        
    # randomly choose products from dataset corresponding to gender, and add them to the user's products bought, viewed and wishlist    
    for j in range(random.randint(1, 15)):
        idx = random.randint(0, len(product_details)-1)
        if "product_id" in product_details.iloc[idx] and product_details.iloc[idx]["in_stock"]:
            products_bought.append([product_details.iloc[idx]["product_id"],product_details.iloc[idx]["name"], product_details.iloc[idx]["share_url"]])
    data["ProductsBought"] = products_bought
    for j in range(random.randint(1, 15)):
        idx = random.randint(0, len(product_details)-1)
        if "product_id" in product_details.iloc[idx] and product_details.iloc[idx]["in_stock"]:
            products_viewed.append([product_details.iloc[idx]["product_id"],product_details.iloc[idx]["name"], product_details.iloc[idx]["share_url"]])
    data["ProductsViewedInLast30Days"] = products_viewed
    for j in range(random.randint(1, 15)):
        idx = random.randint(0, len(product_details)-1)
        if "product_id" in product_details.iloc[idx] and product_details.iloc[idx]["in_stock"]:
            products_wishlist.append([product_details.iloc[idx]["product_id"],product_details.iloc[idx]["name"], product_details.iloc[idx]["share_url"]])
    data["ProductsInWishlist"] = products_wishlist
    users.loc[len(users)] = data

# ensure unique first names in the dataset
names = [fake.unique.first_name() for i in range(100)]

for i in range(100):
    users.loc[i, "FirstName"] = names[i]

   

100%|██████████| 100/100 [00:07<00:00, 13.85it/s]


In [10]:
users.head()

,UserID,FirstName,LastName,Age,Gender,Country,ProductsBought,ProductsViewedInLast30Days,ProductsInWishlist
0,469893,Eric,Allen,29,M,Sierra Leone,"[[BZRGGG26TPY39J4T, BASICS Men Solid Tuxedo St...","[[SGLFZCRCYYPS6HJX, Fair-x UV Protection Aviat...","[[SHOG8U2ZNSMGPZ7R, Aragats Casual Sneakers Fo..."
1,576657,Kevin,Adams,35,M,United States of America,"[[SHOGMMHVCJSXBKMH, SHOEFLY Exclusive Attracti...","[[KTAG79B5XZT6HJJX, JD FINISHING Women Embelli...","[[JEAGC5WUG4EFZHGA, MUFTI Men Slim Mid Rise Gr..."
2,784065,Madeline,Wheeler,61,M,Tonga,"[[KETG93ZYV6GJR9W6, PK Collection Baby Boys We...","[[ERGFMWQ8Q2MK97H3, MEENAZ Fashion Multy jewel...","[[SRWGFNEWHZMQGYN5, PSKS ETHNIC WEAR Embellish..."
3,281469,Richard,Jefferson,39,F,Jamaica,"[[LCHGKQHBGXJ9RMGZ, BVESH Self Design Semi Sti...","[[LCHGKQGDZJMBGCDG, BVESH Self Design Semi Sti...","[[ERGFYP4VDHZFFHSH, MEENAZ Traditional South S..."
4,439629,William,Brown,55,M,Dominica,"[[nan, nan, nan], [SRWG4TW9HF2CQKSF, SEMINO Ki...","[[JEAFYQGSHZYTZP5K, METRONAUT By Flipkart Men ...","[[JEAGJZYBGAHHSGBF, MUFTI Men Slim Mid Rise Gr..."


In [16]:
# save the users dataset as a pickle and excel file
users.to_excel('users.xlsx', index=False)
users.to_pickle('users.pkl')